# Part A

- **Love**: Follow and maintain optimal distance
- **Fear**: Retreat from detected humans
- **Aggressive**: Relentless pursuit
- **Curious**: Cautious investigation with circling

Keyboard controls:
- 'l' = Love behavior
- 'f' = Fear behavior  
- 'a' = Aggressive behavior
- 'c' = Curious behavior
- 's' = Stop

**Step 1: Display Widgets**

Standard widget setup for displaying color and depth images side by side.

In [1]:
import ipywidgets.widgets as widgets
from IPython.display import display
import cv2

# Create two widgets for displaying images
display_color = widgets.Image(format='jpeg', width='45%')
display_depth = widgets.Image(format='jpeg', width='45%')
layout = widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth], layout=layout)
display(sidebyside)

# convert numpy array to jpeg for displaying
def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg', value)[1])

**Step 2: Load YOLO Model**

load the TensorRT YOLO model for human detection.

In [2]:
from ultralytics import YOLO

model = YOLO("yolo11l_half.engine")

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


In [3]:
import cv2
import numpy as np

class HistogramPersonIdentifier:
    """
    Simple HSV-histogram-based person re-identification.
    Keeps an in-memory DB of person_id -> histogram.
    """

    def __init__(self, similarity_thresh=0.6, h_bins=16, s_bins=16):
        self.person_db = {}   # person_id -> hist
        self.next_id = 0
        self.similarity_thresh = similarity_thresh
        self.h_bins = h_bins
        self.s_bins = s_bins

    def _get_histogram(self, roi_bgr):
        """Compute normalised HSV histogram for a ROI."""
        hsv = cv2.cvtColor(roi_bgr, cv2.COLOR_BGR2HSV)
        hist = cv2.calcHist(
            [hsv], [0, 1], None,
            [self.h_bins, self.s_bins],
            [0, 180, 0, 256]
        )
        cv2.normalize(hist, hist, 0, 1, cv2.NORM_MINMAX)
        return hist

    def _match_person(self, hist):
        """Return best-matching person ID or None."""
        best_id = None
        best_score = -1

        for pid, ref_hist in self.person_db.items():
            score = cv2.compareHist(ref_hist, hist, cv2.HISTCMP_CORREL)
            if score > best_score:
                best_score = score
                best_id = pid

        if best_score > self.similarity_thresh:
            return best_id
        return None

    def assign_ids(self, frame, detections):
        """
        For each person box, assign a stable ID based on histogram.
        Returns list of (x1, y1, x2, y2, person_id).
        """
        results = []

        for box in detections.boxes:
            cls = int(box.cls[0])
            if cls != 0:
                continue  # only PERSON class

            x1, y1, x2, y2 = map(int, box.xyxy[0])
            roi = frame[y1:y2, x1:x2]

            if roi.size == 0:
                continue

            hist = self._get_histogram(roi)
            pid = self._match_person(hist)

            # New identity if no good match
            if pid is None:
                pid = self.next_id
                self.person_db[pid] = hist
                self.next_id += 1

            results.append((x1, y1, x2, y2, pid))

        return results

**Step 3: Camera System Initialization**

In [4]:
# ZED camera initialization
import traitlets
import numpy as np
import pyzed.sl as sl
import threading
import motors
from traitlets.config.configurable import SingletonConfigurable

# initialize robot motor control
robot = motors.MotorsYukon(mecanum=False)

# Camera class with traitlets
class Camera(SingletonConfigurable):
    color_value = traitlets.Any()
    
    def __init__(self):
        super(Camera, self).__init__()

        # ZED camera initialization
        self.zed = sl.Camera()
        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.VGA  # 672x376
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA
        init_params.coordinate_units = sl.UNIT.MILLIMETER  # Distance in mm

        # Open camera
        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS:
            print("Camera Open : "+repr(status)+". Exit program.")
            self.zed.close()
            exit(1)

        self.runtime = sl.RuntimeParameters()
        self.thread_runnning_flag = False

        # Get camera resolution
        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        self.image = sl.Mat(self.width, self.height, sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
        self.point_cloud = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C4, sl.MEM.CPU)

    # threaded frame capture
    def _capture_frames(self):
        while(self.thread_runnning_flag == True):
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                # Retrieve images
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
                
                # Convert BGRA to BGR 
                self.color_value_BGRA = self.image.get_data()
                self.color_value = cv2.cvtColor(self.color_value_BGRA, cv2.COLOR_BGRA2BGR)
                # Get depth as numpy array
                self.depth_image = np.asanyarray(self.depth.get_data())
                
    # thread control methods
    def start(self):
        if self.thread_runnning_flag == False:
            self.thread_runnning_flag = True
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()

    def stop(self):
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False
            self.thread.join()
            robot.stop()

# Initialize and start camera
camera = Camera()
camera.start()

[2025-11-21 12:00:01 UTC][ZED][INFO] Logging level INFO
[2025-11-21 12:00:01 UTC][ZED][INFO] Logging level INFO
[2025-11-21 12:00:01 UTC][ZED][INFO] Logging level INFO
[2025-11-21 12:00:02 UTC][ZED][INFO] [Init]  Depth mode: ULTRA
[2025-11-21 12:00:03 UTC][ZED][INFO] [Init]  Camera successfully opened.
[2025-11-21 12:00:03 UTC][ZED][INFO] [Init]  Camera FW version: 1523
[2025-11-21 12:00:03 UTC][ZED][INFO] [Init]  Video mode: VGA@100
[2025-11-21 12:00:03 UTC][ZED][INFO] [Init]  Serial Number: S/N 35129214
Loading yolo11l_half.engine for TensorRT inference...


**Step 4: Behavior Functions**

In [ ]:
# Global variable for current behavior mode
current_behavior = 'stop'

identifier = HistogramPersonIdentifier(similarity_thresh=0.6)

def detect_human(frame):
    global identifier
    
    """
    Human detection and distance calculation
    
    From Tutorial 5:
    - YOLO detection for humans (class 0)
    - Bounding box extraction: result.boxes.xyxy[i]
    - Center calculation: (bbox[0] + bbox[2]) / 2 for x, (bbox[1] + bbox[3]) / 2 for y
    - Distance extraction: camera.depth_image[center_y, center_x]

    """

    results = model(frame, verbose=False)[0]
    
    # confidence threshold for detections
    conf_threshold = 0.5
    human_detected = False
    min_human_human_human_human_human_human_distance = float('inf')
    closest_bbox = None

    tracks = identifier.assign_ids(frame, results)
    if len(tracks) == 0:
        return human_detected, min_distance, closest_bbox
    
    x1, y1, x2, y2, pid = tracks[0]
    center_x = int((x1 + x2) / 2)
    center_y = int((y1 + y2) / 2)
    distance = camera.depth_image[center_y, center_x]
    closest_bbox = [x1, y1, x2, y2]                
    
    return human_detected, min_distance, closest_bbox

def detect_obstacle(depth_image, min_distance):
    depth = np.nan_to_num(depth_image.copy(), nan=0).astype(np.float32)

    # Crop region of interest
    depth[:200, :] = 0
    depth[376:, :] = 0
    depth[:, :168] = 0
    depth[:, 504:] = 0

    # Filter out noise
    depth[(depth < 100) | (depth > 1000)] = 0

    # If nothing valid is detected
    if depth.max() == 0:
        return "no_depth", None

    # Find closest obstacle
    distance = depth[depth != 0].min()

    if distance < min_distance:
        return True, distance
    else:
        return False, distance


def love_behavior(frame, depth_colormap, human_detected, human_distance, bbox):
    """
    Love behavior - follow human and maintain comfortable distance
    
    Distance thresholds chosen based on testing:
    - < 450mm: Too close, back up
    - 450-600mm: Optimal distance, stop
    - 600-1200mm: Medium range, approach slowly
    - > 1200mm: Far away, approach faster
    
    """
    
    if human_detected and human_distance != float('inf'):
        # Draw detection box (Tutorial 5 pattern)
        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), 
                     (int(bbox[2]), int(bbox[3])), (255, 105, 180), 2)
        
        # Calculate if human is left/right of center (Tutorial 5 approach)
        bbox_center_x = (bbox[0] + bbox[2]) / 2
        frame_center_x = camera.width / 2
        
        # Distance-based behavior (new thresholds for love behavior)
        if human_distance < 450:
            robot.backward(0.2)
            cv2.putText(depth_colormap, f'LOVE: Too close {human_distance:.0f}mm', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        elif human_distance >= 450 and human_distance <= 600:
            robot.stop()
            cv2.putText(depth_colormap, f'LOVE: Perfect {human_distance:.0f}mm <3', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
        elif human_distance > 600 and human_distance <= 1200:
            # Medium distance - turn to center, then approach
            # 50 pixel threshold to avoid constant adjustment
            if bbox_center_x < frame_center_x - 50:
                robot.left(0.3) 
            elif bbox_center_x > frame_center_x + 50:
                robot.right(0.3) 
            else:
                robot.forward(0.3) 
            cv2.putText(depth_colormap, f'LOVE: Approaching {human_distance:.0f}mm', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 180, 0), 2)
        else:  # distance > 1200
            # Far away - approach faster with turning
            if bbox_center_x < frame_center_x - 50:
                robot.left(0.4)
            elif bbox_center_x > frame_center_x + 50:
                robot.right(0.4)
            else:
                robot.forward(0.5)
            cv2.putText(depth_colormap, f'LOVE: Coming {human_distance:.0f}mm', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 100, 0), 2)
    else:
        # No human detected - stop and wait
        robot.stop()
        cv2.putText(depth_colormap, 'LOVE: Waiting...', 
                   (200, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (128, 128, 128), 2)
        
def avoid_obstacle(human_detected, obstacle_detected, obstacle_distance):
    global avoid_state  # internal state for the avoidance routine

    if avoid_state == "start":
        robot.turn_left(0.4)
        if not obstacle_detected:
            avoid_state = "following_edge"
        return False  # still avoiding

    elif avoid_state == "following_edge":
        if not obstacle_detected:
            robot.right(0.3)

        elif obstacle_detected:
            robot.forward(0.3)
            if obstacle_distance < 200:
                robot.turn_left(0.3)
        elif human_detected and not obstacle_detected:
            avoid_state = "finished"
        return False

    elif avoid_state == "finished":
        avoid_state = "start"   # reset for next obstacle
        return True              # avoidance is complete

**Step 5: Keyboard Control and Main Callback**

Based on Answer_keyboard_control.ipynb pattern for keyboard input handling.

In [ ]:
text_input = widgets.Text(
    value='',
    placeholder='Type: g=Go, s=Stop',
    description='Behavior:',
    disabled=False
)

# observe text changes
def on_text_change(change):
    """
    Handle keyboard input to switch behaviors
    """
    global current_behavior
    input_value = change['new']
    
    if len(input_value) > 0:
        last_char = input_value[-1].lower()

        # Switch behavior based on input
        if last_char == 'g':
            current_behavior = 'go'
            print('\nSTARTED - Robot started')
        elif last_char == 's':
            current_behavior = 'stop'
            robot.stop()
            print('\nSTOPPED - Robot halted')

text_input.observe(on_text_change, names='value')
display(text_input)

def main_callback(change):
    """
    Main callback function - executes current behavior    
    """
    # Get updated frame
    frame = change['new']
    
    # Create depth colormap for display
    # Converts depth values to color visualization
    depth_image = camera.depth_image
    depth_colormap = cv2.applyColorMap(
        cv2.convertScaleAbs(depth_image, alpha=0.03), 
        cv2.COLORMAP_JET)
    
    # Detect humans and measure distance (Tutorial 5 method)
    human_detected, human_distance, bbox = detect_human(frame)
    obstacle_detected, obstacle_distance = detect_obstacle(depth_image, 400)
    
    # Execute behavior based on current mode
    if current_behavior == 'go':
        if obstacle_detected:
            current_behavior = 'avoid'

        love_behavior(frame, depth_colormap, human_detected, human_distance, bbox)
    elif current_behavior == 'avoid':

        done = avoid_obstacle(obstacle_distance)  

        if done:
            current_behavior = 'go'
            
    elif current_behavior == 'stop':
        robot.stop()
        cv2.putText(depth_colormap, 'STOPPED', 
                   (250, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (128, 128, 128), 2)
    
    # Display current behavior mode on frame
    cv2.putText(frame, f'Mode: {current_behavior.upper()}', 
               (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    # Display images
    scale = 0.3
    resized_color = cv2.resize(frame, None, fx=scale, fy=scale, 
                               interpolation=cv2.INTER_AREA)
    resized_depth = cv2.resize(depth_colormap, None, fx=scale, fy=scale, 
                               interpolation=cv2.INTER_AREA)
    display_color.value = bgr8_to_jpeg(resized_color)
    display_depth.value = bgr8_to_jpeg(resized_depth)

camera.observe(main_callback, names=['color_value'])

print('Press keys to switch modes:')
print('  g = Go (start robot)')
print('  s = Stop (halt robot)')
print('===================================')

Text(value='', description='Behavior:', placeholder='Type: g=Go, s=Stop')

Press keys to switch modes:
  g = Go (start robot)
  s = Stop (halt robot)


**To stop the system:**

In [7]:
# stop camera thread and motors
#camera.stop()
print('System stopped - camera and robot halted')

System stopped - camera and robot halted


## Implementation Notes

### Distance Calculation
Distance is measured by:
1. Getting bounding box from YOLO: `bbox = result.boxes.xyxy[i]` (format: [x1, y1, x2, y2])
2. Calculating center: `center_x = (bbox[0] + bbox[2]) / 2`, `center_y = (bbox[1] + bbox[3]) / 2`
3. Reading depth at center: `distance = camera.depth_image[center_y, center_x]`
4. Depth image returns distance in millimeters


### Behavior Distance Thresholds:

**Love:** 450mm (too close) | 450-600mm (perfect) | 600-1200mm (medium) | >1200mm (far)

**Fear:** 500mm (danger) | 900mm (caution) | 1300mm (alert) | >1300mm (safe)

**Aggressive:** 300mm (ram) | 600mm (engage) | 1000mm (rush) | >1000mm (chase)

**Curious:** 400mm (investigate) | 800mm (approach) | >800mm (interested)